# Report TP1 SD211
### Made by: Jean Paul Saba, Caren Dib

#### Question 3.1

Given:
- $A\mathbf{w} = \mathbf{b}$
- $\mathbf{w} = \begin{bmatrix} \mathbf{w}_1 \\ \mathbf{w}_0 \\ \mathbf{w}_2 \end{bmatrix}$, where $\mathbf{w}_1$ and $\mathbf{w}_2$ are vectors, and $\mathbf{w}_0$ is a scalar.
- $\mathbf{b}_t = y(t)$
- $(A\mathbf{w})_t = \tilde{\mathbf{x}}(t)^T\mathbf{w}_1 + \mathbf{w}_0 - y(t) \tilde{\mathbf{x}}(t)^T\mathbf{w}_2$

We need to show that:
$$ y(t) = \frac{\mathbf{w}_1^T \tilde{\mathbf{x}}(t) + \mathbf{w}_0}{\mathbf{w}_2^T \tilde{\mathbf{x}}(t) + 1} $$

**Proof:**

From $A\mathbf{w}_t = \tilde{\mathbf{x}}(t)^T\mathbf{w}_1 + \mathbf{w}_0 - y(t) \tilde{\mathbf{x}}(t)^T\mathbf{w}_2$, we replace $Aw$ by $b$ which is $y(t)$:

$$y(t) = \tilde{\mathbf{x}}(t)^T\mathbf{w}_1 + \mathbf{w}_0 - y(t) \tilde{\mathbf{x}}(t)^T\mathbf{w}_2$$

we rearrange to isolate $y(t)$:

$$ y(t) = \frac{\tilde{\mathbf{x}}(t)^T\mathbf{w}_1 + \mathbf{w}_0}{\tilde{\mathbf{x}}(t)^T\mathbf{w}_2 + 1} $$

and as $A^T B = B^T A$, then:

$$ y(t) = \frac{\mathbf{w}_1^T\tilde{\mathbf{x}}(t) + \mathbf{w}_0}{\mathbf{w}_2^T\tilde{\mathbf{x}}(t) + 1} $$

This is the required form of $y(t)$. Therefore, we have shown that if $A\mathbf{w} = \mathbf{b}$, then $y(t)$ is as stated in the question.


#### Question 3.2

In [1]:
import numpy as np

data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
print(x)

[-0.00927821  0.08309371 -0.03672704 ...  0.01980595 -0.03057174
 -0.01188614]


#### Question 3.3

In [2]:
M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]

In [3]:
from sklearn.metrics import mean_squared_error
y_pred = A_test.dot(x)
mse = mean_squared_error(b_test, y_pred)
mse

780.8984793523193

#### Question 3.4

In [4]:
from sklearn.linear_model import Ridge

ridge = Ridge(100)
ridge.fit(A, b)
new_b = ridge.predict(A_test)
new_mse = mean_squared_error(b_test, new_b)
new_mse

63.71840515652645

The mean squared error after using the ridge model (regularized) is less than the unregularized one.

#### Question 3.5

$f1: w \rightarrow \frac{1}{2}||Aw-b||^2 + \frac{\lambda}{2}||w||^2$

Let's break it down into two parts:
* $\nabla(\frac{1}{2}||Aw-b||^2) = A^T(Aw-b)$
* $\nabla(\frac{\lambda}{2}||w||^2)=\lambda w$
so:

$\nabla f_1(w)=A^T(Aw-b) + \lambda w$

Let's calculate the second derivative of $f_1(w)$ to check the convexivity.

$\nabla ^2 f_1(w)=A^T A + \lambda I$

So $f_1(w)$ is convex if $A^T A + \lambda I$ is positive semidefinite. 

#### Question 3.6

In [5]:
def gradient_f1(w, A, b, lambda_reg):
    term1 = A.T.dot(A.dot(w) - b)
    term2 = lambda_reg * w
    return term1 + term2

def lipschitz_constant(A):
    eigenvalues = np.linalg.eigvals(np.dot(A.T, A))
    L = np.max(np.real(eigenvalues))
    return L

def gradient_descent(A, b, lambda_reg, gradient_tol):
    alpha = 1 / lipschitz_constant(A)
    print("Step size: ", alpha)
    w = np.zeros(A.shape[1])
    iterations = 0
    norm_gradient = float('+inf')
    
    while norm_gradient > gradient_tol:
        gradient = gradient_f1(w, A, b, lambda_reg)
        norm_gradient = np.linalg.norm(gradient)
        
        w -= alpha * gradient
        iterations += 1
    
    return w, iterations

lambda_reg = 100
gradient_tol = 1

w_optimal, num_iterations = gradient_descent(A, b, lambda_reg, gradient_tol)

norm_gradient_optimal = np.linalg.norm(gradient_f1(w_optimal, A, b, lambda_reg))

(w_optimal, num_iterations, norm_gradient_optimal)


Step size:  2.863243114925734e-07


(array([-0.01238055,  0.05775866, -0.00111774, ...,  0.01579912,
        -0.03571617,  0.0133528 ]),
 97544,
 0.9999369501661834)

I chose $ \frac{1}{L} $ constant as the constant step size. The Lipschitz constant is employed in optimization algorithms, such as gradient descent, to ensure stability and convergence. In the context of convex optimization problems, the Lipschitz continuity of the gradient is a crucial property.

For gradient descent, denoted as $L$, the Lipschitz constant is related to the smoothness of the objective function. The Lipschitz condition for the gradient is expressed as:

$$ ||\nabla f(x) - \nabla f(y)|| \leq L ||x - y|| $$

where $f$ is the objective function, $\nabla f(x)$ is the gradient of $f$ at point $x$, and $L$ is the Lipschitz constant. This condition bounds the rate at which the gradient can change across different points in the function space.

In the provided code for gradient descent, the Lipschitz constant is used to set a step size ($\alpha$) that ensures convergence without oscillations or divergence. The Lipschitz constant acts as an upper bound on the magnitude of the gradient, and choosing the step size inversely proportional to $L$ helps balance convergence speed and stability.

It took 97k iterations to get to under 1.

#### Question 4.1

Let's define the differentiable function $ f2(w) $ and the non-smooth, convex function $ g2(w) $:

$$ f2(w) = \frac{1}{2} \|Aw - b\|_2^2 $$

$$ g2(w) = \lambda \|w\|_1 $$

The objective function can then be expressed as the sum of $ f2(w) $ and $ g2(w) $:

$$ F2(w) = f2(w) + g2(w) $$

The gradient of $ f2(w) $ is given by:

$$ \nabla f2(w) = A^T(Aw - b) $$

The proximal operator of $ g2(w) $, denoted as $ \text{prox}_{g2}(v) $, is the soft-thresholding operator:

$$ \text{prox}_{g2}(v) = \text{sign}(v) \cdot \max(0, |v| - \lambda) $$

where $ \text{sign}(v) $ is the sign function and $ \lambda $ corresponds to the regularization parameter in the objective function.

#### Question 4.2

In [12]:
import numpy as np

def soft_thresholding_operator(v, threshold):
    return np.sign(v) * np.maximum(0, np.abs(v) - threshold)

def proximal_gradient_method(A, b, lambda_val, epsilon=1e-6):
    m, n = A.shape
    w = np.zeros((n, 1))
    alpha = 1 / lipschitz_constant(A)
    
    gradient = np.dot(A.T, np.dot(A, w) - b)
    norm_gradient = np.linalg.norm(gradient, ord=2) + 1000

    while norm_gradient > epsilon:
        gradient = np.dot(A.T, np.dot(A, w) - b)
        w = w - alpha * gradient

        w = soft_thresholding_operator(w, lambda_val * alpha)

        new_norm_gradient = np.linalg.norm(gradient, ord=2)

        if np.abs(new_norm_gradient - norm_gradient) < epsilon:
            break
        
        norm_gradient = new_norm_gradient

    return w

lambda_val = 200

result = proximal_gradient_method(A, b, lambda_val, epsilon=100)
print("Optimal w:", result)

Optimal w: [[-0.         -0.         -0.         ... -0.         -0.
  -0.        ]
 [ 0.03048354  0.03088203  0.00500326 ...  0.0079661   0.0026795
   0.0064129 ]
 [ 0.00750956  0.00770429  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.         -0.         ... -0.         -0.
  -0.        ]
 [-0.00334578 -0.00353691  0.         ...  0.          0.
   0.        ]
 [ 0.          0.         -0.         ... -0.         -0.
  -0.        ]]


I suggest testing when the norm isn't decreasing much to stop.

#### Question 4.3

In [22]:
def proximal_gradient_method_line_search(A, b, lambda_reg, tol=1e-4):
    n_features = A.shape[1]
    w = np.zeros(n_features)
    alpha = 1.0 / lipschitz_constant(A)

    while True:
        gradient_smooth = np.dot(A.T, (np.dot(A, w) - b))
        w_new = soft_thresholding_operator(w - alpha * gradient_smooth, alpha * lambda_reg)

        while f2(A, b, w_new, lambda_reg) > f2(A, b, w, lambda_reg) + np.sum(gradient_smooth * (w_new - w)) + (1 / (2 * alpha)) * np.linalg.norm(w_new - w, 2) ** 2:
            alpha /= 2

        if np.linalg.norm(w_new - w, 2) < tol:
            break

        w = w_new

    return w

def f2(A, b, w, lambda_reg):
    residual = np.dot(A, w) - b
    return 0.5 * np.linalg.norm(residual, 2) ** 2 + lambda_reg * np.linalg.norm(w, 1)

lambda_reg = 200

result = proximal_gradient_method_line_search(A, b, lambda_reg)
print(result)

[-0.          0.00166146  0.00091599 ... -0.          0.00127137
 -0.00195512]


The algorithm with fixed step size takes ages to converge, but the one with line search takes couple of seconds.